In [ ]:
#  https://github.com/rdkit/rdkit-tutorials/blob/master/notebooks/002_SMARTS_SubstructureMatching.ipynb

In [ ]:
from __future__ import print_function
# import rdkit components
from rdkit import rdBase
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw

# use IPythonConsole for pretty drawings
from rdkit.Chem.Draw import IPythonConsole

In [ ]:
#defining the variables as my compounds, and adding them in a list

naphthalene = Chem.MolFromSmiles('c12ccccc1cccc2')
benzene = Chem.MolFromSmiles('c1ccccc1')
benzoxazole = Chem.MolFromSmiles('n1c2ccccc2oc1')
indane = Chem.MolFromSmiles('c1ccc2c(c1)CCC2')
skatole = Chem.MolFromSmiles('CC1=CNC2=CC=CC=C12')
quinoline = Chem.MolFromSmiles('n1cccc2ccccc12')

my_compounds = [naphthalene,benzene,benzoxazole,indane,skatole,quinoline]

In [ ]:
#To view the molecules 

Draw.MolsToGridImage(my_compounds, useSVG=False)

In [ ]:
""" Check https://www.daylight.com/dayhtml_tutorials/languages/smarts/#:~:text=and%20aromatic%20atoms.-,%5BR2%5D%20%C2%A0,-Atoms%20that%20are
[R2] in SMARTS means the list of atoms that are a part of 2 rings
SSSR = smallest set of smallest rings ( https://depth-first.com/articles/2020/08/31/a-smallest-set-of-smallest-rings/ )

This code tests whether any atom in each molecule is a member of two rings 

""" 
is_in_two_rings = Chem.MolFromSmarts('[R2]')
[mol.HasSubstructMatch(is_in_two_rings) for mol in my_mos]

In [ ]:
#For RDKit molecules created via SMILES, ring properties are instantiated automatically on molecule creation.
#Using SMARTS instead of SMILES to initialize molecules does not result in instantiation of a molecule's RingInfo property. Thus, until Chem.GetSSSR() is invoked on a SMARTS-derived molecule, certain queries will fail.

In [ ]:
benzo_five = Chem.MolFromSmarts('[*r5R1]1[cR2]2[cR1][cR1][cR1][cR1][cR2]2[*r5R1][*r5R1]1')
benzo_six = Chem.MolFromSmarts('[*r6R1]1[cR2]2[cR1][cR1][cR1][cR1][cR2]2[*r6R1][*r6R1][*r6R1]1')
smarts_queries = [benzo_five, benzo_six]

In [ ]:
# our first try fails
try:
    print([mol.HasSubstructMatch(is_in_two_rings) for mol in smarts_queries])
    print('Success!  Your query worked.')
except RuntimeError as err:
    print('ERROR; {0} because RingInfo wasn\'t initialized\n'.format(err))

In [ ]:
# initialize rings
[Chem.GetSSSR(mol) for mol in smarts_queries]

# now it works
try:
    print([mol.HasSubstructMatch(is_in_two_rings) for mol in smarts_queries])
    print('Success!  Your query worked.\n')
except RuntimeError as err:
    print('ERROR; {0} because RingInfo wasn\'t initialized'.format(err))
    
    #(doubt. what are we trying to do here?)

In [ ]:
"""RingInfo is instantiated, it will be possible to compute meaningful 2D sets of coordinates for the SMARTS molecule and to both (a) query against the SMARTS molecule and (b) use the SMARTS molecule as a query against other molecules. Without calling Chem.GetSSSR(), only the second option would be viable."""

In [ ]:
[AllChem.Compute2DCoords(mol) for mol in smarts_queries]

In [ ]:
# matching the first query
for mol in my_compounds:
    AllChem.GenerateDepictionMatching2DStructure(mol, benzo_five, acceptFailure = True)

In [ ]:
#matching to our first query is done. Now we have to draw the results

In [ ]:
# IPythonConsole module of rdkit.Chem.Draw doesn't seem to pass through the highlightSubstructs = True parameter to the appropriate rendering function, even though it does result in setting the mol.__sssAtoms property for each molecule. We can pass this parameter manually.

In [ ]:
highlight_lists = [mol.GetSubstructMatch(benzo_five) for mol in my_compounds]
Draw.MolsToGridImage(my_compounds, 
                     highlightAtomLists = highlight_lists,
                     useSVG=False
                    )

In [ ]:
"""skatole, indane, and benzoxazole hit our first query, which tested whether a molecule contained a benzene fused to a five-membered ring. The skatole example illustrates that not all atoms in a molecule need to match a query, and also that only those atoms that match will be highlighted. benzene isn't highlighted because it doesn't contain two rings, and naphthalene and quinoline aren't highlighted because they contain benzene fused to a six-membered ring."""

In [ ]:
for mol in my_compounds:
    AllChem.GenerateDepictionMatching2DStructure(mol, benzo_six, acceptFailure = True)
# make sure the molecules are storing information about the SSS match:
highlight_lists = [mol.GetSubstructMatch(benzo_six) for mol in my_compounds]
Draw.MolsToGridImage(my_compounds, 
                     highlightAtomLists = highlight_lists,
                     useSVG=False
                    )